In [2]:
import re, string,os
from glob import glob as gb
import pandas as pd
from tqdm import tqdm
from collections import Counter
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib
from functions import *

base_path = "/media/ruben/OSDisk/Users/ruben.ros/Documents/GitHub/ParlaMintCase"

In [4]:
df = data_loader.full('si','lemmatized')
df = utils.add_metadata(df,'si')


In [5]:
for i in set(df['speaker_party_name']):
    print(i)

Stranka Alenke Bratušek
Slovenska demokratska stranka
Zavezništvo Alenke Bratušek
Slovenska nacionalna stranka
Demokratična stranka upokojencev Slovenije
Stranka modernega centra
Poslanska skupina nepovezanih poslancev
Združena levica
Nova Slovenija – Krščanski demokrati
Socialni demokrati
Stranka Mira Cerarja
Lista Marjana Šarca
Nepovezani poslanci
Poslanci italijanske in madžarske narodne skupnosti
_
Levica


In [11]:
dict(zip(df['speaker_party'],df['speaker_party_name']))

{'PiS': 'Klub Parlamentarny Prawo i Sprawiedliwość',
 '_': '_',
 'KO': 'Klub Parlamentarny Koalicja Obywatelska - Platforma Obywatelska, Nowoczesna, Inicjatywa Polska, Zieloni',
 'KP-PSL': 'Klub Parlamentarny Koalicja Polska - Polskie Stronnictwo Ludowe - Kukiz15',
 'Niezrzeszeni': 'Posłowie i senatorowie niezrzeszeni',
 'Przywrócić Prawo': 'Koło Poselskie Przywrócić Prawo',
 "Kukiz'15": "Klub Parlamentarny Kukiz'15",
 'Konfederacja': 'Konfederacja',
 'Teraz!': 'Koło Poselskie Teraz!',
 'UPR': 'Koło Poselskie Unia Polityki Realnej',
 'KO;Niezrzeszeni': 'Klub Parlamentarny Koalicja Obywatelska - Platforma Obywatelska, Nowoczesna, Inicjatywa Polska, Zieloni;Posłowie i senatorowie niezrzeszeni',
 'PiS;Niezrzeszeni': 'Klub Parlamentarny Prawo i Sprawiedliwość;Posłowie i senatorowie niezrzeszeni',
 'PiS;KO;Niezrzeszeni': 'Klub Parlamentarny Prawo i Sprawiedliwość;Klub Parlamentarny Koalicja Obywatelska - Platforma Obywatelska, Nowoczesna, Inicjatywa Polska, Zieloni;Posłowie i senatorowie ni

In [12]:
df

,id,text,subcorpus,speaker_birth,segments,speaker_party_name,speaker_party,speaker_role,tokens,speaker_gender,speaker_id,session,speaker_type,term,speaker_name,words,sitting,sentences,date,names
0,ParlaMint-PL_2017-01-26-sejm-34-4.u1,wznawiam posiedzenie na sekretarzy dzisiejszyc...,Reference,1949,7,Klub Parlamentarny Prawo i Sprawiedliwość,PiS,chair,68,M,TerleckiRyszard,34-lower,MP,8-lower,"Terlecki, Ryszard",58,?,7,2017-01-26,8
1,ParlaMint-PL_2017-01-26-sejm-34-4.u2,informuję że w dniu dzisiejszym odbędą się pos...,Reference,1984,32,Klub Parlamentarny Prawo i Sprawiedliwość,PiS,regular,348,M,KubówKrzysztof,34-lower,MP,8-lower,"Kubów, Krzysztof",215,?,32,2017-01-26,16
2,ParlaMint-PL_2017-01-26-sejm-34-4.u3,bardzo dziękuję prezydium sejmu w uzgodnieniu ...,Reference,1949,11,Klub Parlamentarny Prawo i Sprawiedliwość,PiS,chair,200,M,TerleckiRyszard,34-lower,MP,8-lower,"Terlecki, Ryszard",172,?,14,2017-01-26,11
3,ParlaMint-PL_2017-01-26-sejm-34-4.u4,bardzo dziękuję panie marszałku panie ministrz...,Reference,1967,4,Klub Parlamentarny Prawo i Sprawiedliwość,PiS,regular,244,F,BorowiakJoanna,34-lower,MP,8-lower,"Borowiak, Joanna",213,?,17,2017-01-26,10
4,ParlaMint-PL_2017-01-26-sejm-34-4.u5,dziękuję bardzo proszę o udzielenie odpowiedzi...,Reference,1949,3,Klub Parlamentarny Prawo i Sprawiedliwość,PiS,chair,25,M,TerleckiRyszard,34-lower,MP,8-lower,"Terlecki, Ryszard",21,?,3,2017-01-26,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331039,ParlaMint-PL_2020-06-17-senat-12-1.u320,dziękuję bardzo panie senatorze informuję że l...,COVID,1972,8,Klub Parlamentarny Koalicja Polska - Polskie S...,KP-PSL,chair,126,M,KamińskiMichał,12-upper,MP,10-upper,"Kamiński, Michał",100,?,9,2020-06-17,10
331040,ParlaMint-PL_2020-06-17-senat-12-1.u321,ja jakby w tym samym duchu co wcześniej,COVID,?,1,_,_,guest,9,M,NiedużakMarek,12-upper,notMP,10-upper,"Niedużak, Marek",8,?,1,2020-06-17,0
331041,ParlaMint-PL_2020-06-17-senat-12-1.u322,tak jest dziękuję bardzo zamykam dyskusję info...,COVID,1972,5,Klub Parlamentarny Koalicja Polska - Polskie S...,KP-PSL,chair,36,M,KamińskiMichał,12-upper,MP,10-upper,"Kamiński, Michał",27,?,7,2020-06-17,2
331042,ParlaMint-PL_2020-06-17-senat-12-1.u323,dziękuję bardzo wysoka izbo komunikat odnośnie...,COVID,1972,8,Klub Parlamentarny Koalicja Obywatelska - Plat...,KO,regular,253,F,KołaczLeszczyńskaAgnieszka,12-upper,MP,10-upper,"Kołacz-Leszczyńska, Agnieszka",228,?,11,2020-06-17,19
